In [1]:
import requests
import time
from dotenv import load_dotenv
import os

# .env 파일에서 환경 변수 불러오기
load_dotenv()

restaurants = []  # 배열 선언

# 전체 페이지 수를 설정
total_pages = 576
url_template = os.getenv("URL_TEMPLATE")
headers = {
    "User-Agent": os.getenv("USER_AGENT"),
    "Accept": os.getenv("ACCEPT"),
    "x-requested-with": os.getenv("X_REQUESTED_WITH")
}

# 크롤링 진행
for page in range(total_pages):
    url = url_template.format(page=page)
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        restaurants.extend(data["_embedded"]["restaurants"])
        print(f"Page {page + 1}/{total_pages} collected successfully.")
    else:
        print(f"Failed to fetch page {page + 1}. Status code: {response.status_code}")
    
    # 각 요청 사이에 시간 간격 추가
    time.sleep(2)  # 2초 간격

# 전체 데이터를 저장
import json
with open("data/restaurants.json", "w", encoding="utf-8") as f:
    json.dump(restaurants, f, ensure_ascii=False, indent=4)

print("전체 페이지 크롤링 완료. JSON 파일로 저장됨.")


Page 1/576 collected successfully.
Page 2/576 collected successfully.
Page 3/576 collected successfully.
Page 4/576 collected successfully.
Page 5/576 collected successfully.
Page 6/576 collected successfully.
Page 7/576 collected successfully.
Page 8/576 collected successfully.
Page 9/576 collected successfully.
Page 10/576 collected successfully.
Page 11/576 collected successfully.
Page 12/576 collected successfully.
Page 13/576 collected successfully.
Page 14/576 collected successfully.
Page 15/576 collected successfully.
Page 16/576 collected successfully.
Page 17/576 collected successfully.
Page 18/576 collected successfully.
Page 19/576 collected successfully.
Page 20/576 collected successfully.
Page 21/576 collected successfully.
Page 22/576 collected successfully.
Page 23/576 collected successfully.
Page 24/576 collected successfully.
Page 25/576 collected successfully.
Page 26/576 collected successfully.
Page 27/576 collected successfully.
Page 28/576 collected successfully.
P

In [2]:
#### 1. 전처리

import pandas as pd
import json

# JSON 파일 읽기
with open("data/restaurants.json", "r", encoding="utf-8") as f:
    restaurants = json.load(f)

# Pandas DataFrame으로 변환
df = pd.DataFrame(restaurants)

# 기존 전처리 코드 적용
# 필요 없는 컬럼 제거
columns_to_drop = ["createdDate", "timeInfo", "gps", "tags", "status",
                   "buzUsername", "business", "pageView", "brandMatchStatus", "brandRejectReason",
                   "orderDescending", "foodTypeDetails","foodDetailTypes", "countEvaluate", "bookmark", "features",
                   "feature107", "brandBranches", "brandHead", "firstImage", "firstLogoImage",
                   "_links"]
df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])

# Nested JSON 컬럼 정규화
nested_columns = ["headerInfo", "defaultInfo", "statusInfo", "juso", "review", "etcInfo"]
for column in nested_columns:
    if column in df.columns:
        n = pd.json_normalize(df[column])  # 정규화
        n.columns = [f"{column}_{subcol}" for subcol in n.columns]  # 열 이름 접두사 추가
        df = pd.concat([df.drop(columns=[column]), n], axis=1)  # 기존 열 삭제 후 결합
if "foodTypes" in df.columns:
    df["foodTypes"] = df["foodTypes"].apply(lambda x: ", ".join(x) if isinstance(x, list) else None)
else:
    print("foodTypes 컬럼이 누락되었습니다. 데이터 확인 필요.")
# 필요 없는 sub_columns 제거
sub_columns_to_drop = ["headerInfo_nickname", "headerInfo_year", "headerInfo_ribbonTypeByOrdinal", "headerInfo_nameEN", "headerInfo_nameCN", "headerInfo_bookYear",
                       "defaultInfo_websiteFacebook", "defaultInfo_chefName", "statusInfo_storeType", "statusInfo_openEra", "statusInfo_openDate",

                       "juso_roadAddrPart2", "juso_jibunAddr", "juso_zipNo",
                       "juso_admCd", "juso_bdNm", "juso_buldMnnm", "juso_buldSlno", "juso_detBdNmList", "juso_zone2_1", "juso_zone2_2", "juso_map_1",
                       "juso_map_2", "juso_detailAddress" , "juso_emdNm", "juso_engAddr", "juso_liNm",  
                       "review_readerReview", "review_businessReview", "review_editorReview",
                       "etcInfo_toilet", "etcInfo_toiletEtc", "etcInfo_close", "etcInfo_interior",
                       "etcInfo_appYn", "etcInfo_projectNo", "etcInfo_reviewerRecommend", "etcInfo_onlySiteView",
                       "etcInfo_history", "etcInfo_mainMemo"]
df = df.drop(columns=[col for col in sub_columns_to_drop if col in df.columns])

# 데이터 저장
df.to_csv("data/blueRibbon.csv", index=False)
print("전처리 테스트 완료. CSV 파일 저장됨.")


전처리 테스트 완료. CSV 파일 저장됨.


In [3]:
data = pd.read_csv("data/blueRibbon.csv")
print(data.columns)
data.iloc[20:40, 0:5]

Index(['id', 'bookStatus', 'foodTypes', 'headerInfo_nameKR',
       'headerInfo_ribbonType', 'defaultInfo_website',
       'defaultInfo_websiteInstagram', 'defaultInfo_phone',
       'defaultInfo_openHours', 'defaultInfo_closeHours',
       'defaultInfo_openHoursWeekend', 'defaultInfo_closeHoursWeekend',
       'defaultInfo_dayOff', 'defaultInfo_app2Yn', 'statusInfo_parking',
       'statusInfo_creditCard', 'statusInfo_visit', 'statusInfo_menu',
       'statusInfo_priceRange', 'statusInfo_businessHours',
       'statusInfo_newOpenDate', 'juso_roadAddrPart1', 'juso_siNm',
       'juso_sggNm', 'juso_rn', 'review_review', 'review_reviewSimple',
       'review_readerAppraisal', 'etcInfo_seat', 'etcInfo_room',
       'etcInfo_chain', 'etcInfo_renewal', 'etcInfo_head', 'etcInfo_branch'],
      dtype='object')


C:\Users\qkrck\AppData\Local\Temp\ipykernel_15476\4223554085.py:1: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("data/blueRibbon.csv")


,id,bookStatus,foodTypes,headerInfo_nameKR,headerInfo_ribbonType
20,29206,APPROVAL,"일식, 모던재패니즈, 컨템포러리",고료리켄,RIBBON_THREE
21,28236,APPROVAL,"한식(육류), 한우오마카세, 소고기구이",본앤브레드,RIBBON_THREE
22,28229,APPROVAL,컨템포러리,알라프리마,RIBBON_THREE
23,28167,APPROVAL,"한식(일반한식), 모던한식",권숙수,RIBBON_THREE
24,26017,APPROVAL,"컨템포러리, 프랑스식",제로컴플렉스,RIBBON_THREE
25,20038,APPROVAL,이탈리아식,테이블포포,RIBBON_THREE
26,15931,APPROVAL,프랑스식,파씨오네,RIBBON_THREE
27,4953,APPROVAL,프랑스식,비스트로드욘트빌,RIBBON_THREE
28,4945,APPROVAL,"한식(일반한식), 모던한식",무궁화,RIBBON_THREE
29,4667,APPROVAL,"한식(일반한식), 모던한식, 뉴코리안",정식당,RIBBON_THREE


In [4]:
import numpy as np
import re

# 빈칸이나 "없음"으로 표기된 칸을 None으로 변경
df.replace(["", "없음"], None, inplace=True)

# foodDetailTypes의 리스트 해제
# 각 행의 값이 리스트라면 이를 문자열로 변환 (쉼표로 구분된 문자열로 병합)
if "foodTypes" in df.columns:
    df["foodTypes"] = df["foodTypes"].apply(lambda x: ", ".join(x) if isinstance(x, list) else x)


# 웹사이트 열 통합
if "defaultInfo_website" in df.columns and "defaultInfo_websiteInstagram" in df.columns:
    def merge_websites(row):
        websites = [site for site in [row.get("defaultInfo_website"), row.get("defaultInfo_websiteInstagram")] if site]
        return ", ".join(websites) if websites else None

    df["defaultInfo_website_combined"] = df.apply(merge_websites, axis=1)
    df.drop(columns=["defaultInfo_website", "defaultInfo_websiteInstagram"], inplace=True)
    df.rename(columns={"defaultInfo_website_combined": "defaultInfo_website"}, inplace=True)

# statusInfo_openDate 형식 변환 (년도 4자리로 추출 후 "2024년" 형식으로 표기)
def extract_year_with_suffix(date):
    if isinstance(date, str):
        # 패턴 매칭으로 숫자 4자리 추출
        match = re.search(r'\d{4}', date)
        if match:
            return f"{int(match.group(0))}년"  # "2024년" 형식으로 반환
    return None

if "statusInfo_openDate" in df.columns:
    df["statusInfo_openDate"] = df["statusInfo_openDate"].apply(extract_year_with_suffix)

# 최종 결과를 CSV 파일로 저장
df.to_csv("data/cleaned_all_restaurants.csv", index=False)
print("수정된 CSV 파일 저장 완료!")


수정된 CSV 파일 저장 완료!


In [5]:
temp = pd.read_csv("data/cleaned_all_restaurants.csv")
temp

C:\Users\qkrck\AppData\Local\Temp\ipykernel_15476\2943191621.py:1: DtypeWarning: Columns (8,9,27) have mixed types. Specify dtype option on import or set low_memory=False.
  temp = pd.read_csv("data/cleaned_all_restaurants.csv")


,id,bookStatus,foodTypes,headerInfo_nameKR,headerInfo_ribbonType,defaultInfo_phone,defaultInfo_openHours,defaultInfo_closeHours,defaultInfo_openHoursWeekend,defaultInfo_closeHoursWeekend,...,review_review,review_reviewSimple,review_readerAppraisal,etcInfo_seat,etcInfo_room,etcInfo_chain,etcInfo_renewal,etcInfo_head,etcInfo_branch,defaultInfo_website
0,4627,APPROVAL,"중식, 일반중식",홍보각,RIBBON_THREE,02-531-6479,NaN,NaN,NaN,NaN,...,국내 중식의 대가인 여경래 셰프가 2023년 이전하여 오픈한 중식 파인 다이닝 레스...,국내 중식의 대가인 여경래 셰프가 2023년 이전하여 오픈한 중식 파인 다이닝 레스...,NaN,64,"6개(2, 3, 4, 6, 8, 12석)",False,False,NaN,NaN,https://www.instagram.com/hongbogak_official/
1,29083,APPROVAL,"컨템포러리, 이노베이티브퀴진, 분자요리, 이탈리아식",쵸이닷,RIBBON_THREE,02-518-0318,12:00,22:30,NaN,NaN,...,최현석 셰프의 크리에이티브한 요리를 즐길 수 있는 레스토랑. 분자요리와 이탈리아 요...,최현석 셰프의 크리에이티브한 요리를 즐길 수 있는 레스토랑. 분자요리와 이탈리아 요...,“어디까지 발전할 수 있을지 진심으로 기대되는 곳.” “음식 너무나 재밌었어요.”,NaN,NaN,False,False,NaN,NaN,https://www.instagram.com/choidot_official/
2,28946,APPROVAL,프랑스식,더그린테이블,RIBBON_THREE,02-591-2672,NaN,NaN,NaN,NaN,...,김은희 셰프가 운영하는 프렌치 레스토랑. 사계절에 따라 변화하는 우리나라 식재료로 ...,김은희 셰프가 운영하는 프렌치 레스토랑. 사계절에 따라 변화하는 우리나라 식재료로 ...,“음식의 결 하나하나에서 섬세한 정성이 듬뿍 느껴진다. 신선한 재료들이 훌륭하게 조...,NaN,NaN,False,False,NaN,NaN,"http://thegreentable.co.kr/, https://www.insta..."
3,4963,APPROVAL,"한식(육류), 소갈비, 소고기구이, 한우오마카세",명월관,RIBBON_THREE,02-6330-9050,NaN,NaN,NaN,NaN,...,"워커힐호텔 내 한옥 분위기의 별채로 있는, 주변 경치가 아름다운 숯불구이 전문점. ...","워커힐호텔 내 한옥 분위기의 별채로 있는, 주변 경치가 아름다운 숯불구이 전문점. ...",NaN,300,4개,False,False,NaN,NaN,https://www.walkerhill.com/grandwalkerhillseou...
4,859,APPROVAL,프랑스식,콘티넨탈,RIBBON_THREE,02-2230-3369,NaN,NaN,NaN,NaN,...,"정통 유럽 스타일의 프렌치 레스토랑으로, 40여 년의 역사를 자랑하는, 서울에서 오...","정통 유럽 스타일의 프렌치 레스토랑으로, 40여 년의 역사를 자랑하는, 서울에서 오...",“최고의 레스토랑이다.” “분위기가 상당히 모던하고 세련된 느낌. 재료의 질이 최상...,70석,1개,False,False,NaN,NaN,https://www.shilla.net/seoul/dining/viewDining...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17275,25891,INACTIVE,"한식(어패류), 생선회",8호단골횟집,NOT_RECORD,032-885-9955,10:00,22:00,NaN,NaN,...,다양한 해산물을 저렴한 가격에 모두 맛볼 수 있는 곳. 스페셜 메뉴를 주문하면 한상...,다양한 해산물을 저렴한 가격에 모두 맛볼 수 있는 곳. 스페셜 메뉴를 주문하면 한상...,NaN,NaN,NaN,False,False,NaN,NaN,NaN
17276,26401,INACTIVE,"일식, 스시",키무스시,NOT_RECORD,031-411-0603,12:00,22:00,NaN,NaN,...,아담한 크기의 매장에서 스시를 즐길 수 있는 곳. 주문을 하면 일본인 주방장이 바로...,아담한 크기의 매장에서 스시를 즐길 수 있는 곳. 주문을 하면 일본인 주방장이 바로...,NaN,NaN,NaN,False,False,NaN,NaN,NaN
17277,26400,INACTIVE,"카페/커피/티, 카페",카페바다,NOT_RECORD,010-3713-3256,10:00,19:00,NaN,NaN,...,안면도 밧개 해변 근처에서 핸드드립커피를 즐길 수 있는 곳. 아기자기하고 아늑한 인...,안면도 밧개 해변 근처에서 핸드드립커피를 즐길 수 있는 곳. 아기자기하고 아늑한 인...,NaN,NaN,NaN,False,False,NaN,NaN,"https://cafebada.modoo.at/, https://www.instag..."
17278,26203,INACTIVE,"디저트/베이커리, 떡",미시루,NOT_RECORD,042-825-9902,06:00,19:00,NaN,NaN,...,"현대적 감각의 맛과 멋, 영양까지 골고루 갖춘 떡을 맛볼 수 있는 곳. 100% 천...","현대적 감각의 맛과 멋, 영양까지 골고루 갖춘 떡을 맛볼 수 있는 곳. 100% 천...",NaN,NaN,NaN,False,False,NaN,진관점(02-388-7710),http://www.미시루.com/


In [6]:
temp.columns

Index(['id', 'bookStatus', 'foodTypes', 'headerInfo_nameKR',
       'headerInfo_ribbonType', 'defaultInfo_phone', 'defaultInfo_openHours',
       'defaultInfo_closeHours', 'defaultInfo_openHoursWeekend',
       'defaultInfo_closeHoursWeekend', 'defaultInfo_dayOff',
       'defaultInfo_app2Yn', 'statusInfo_parking', 'statusInfo_creditCard',
       'statusInfo_visit', 'statusInfo_menu', 'statusInfo_priceRange',
       'statusInfo_businessHours', 'statusInfo_newOpenDate',
       'juso_roadAddrPart1', 'juso_siNm', 'juso_sggNm', 'juso_rn',
       'review_review', 'review_reviewSimple', 'review_readerAppraisal',
       'etcInfo_seat', 'etcInfo_room', 'etcInfo_chain', 'etcInfo_renewal',
       'etcInfo_head', 'etcInfo_branch', 'defaultInfo_website'],
      dtype='object')

In [7]:
import pandas as pd

# 1. 데이터 로드
file_path = 'data/cleaned_all_restaurants.csv'
restaurant_data = pd.read_csv(file_path)

# 2. 컬럼명 변경 매핑
# review_review    review_reviewSimple    review_readerAppraisal    etcInfo_seat    etcInfo_room
column_rename_mapping = {
        'foodTypes': '음식종류',
        'headerInfo_nameKR': '식당이름',
        'headerInfo_ribbonType': '리본개수',
    'defaultInfo_phone': '전화번호',
    'defaultInfo_openHours': '영업시간_평일',
    'defaultInfo_closeHours': '영업종료_평일',
    'defaultInfo_openHoursWeekend': '영업시간_주말',
    'defaultInfo_closeHoursWeekend': '영업종료_주말',
    'defaultInfo_dayOff': '휴무일',
    'defaultInfo_app2Yn': '앱사용여부',
    'defaultInfo_website': '웹사이트',
    'statusInfo_parking': '주차가능여부',
    'statusInfo_creditCard': '카드결제가능여부',
    'statusInfo_visit': '방문가능여부',
    'statusInfo_menu': '메뉴정보',
    'statusInfo_priceRange': '가격대',
    'statusInfo_businessHours': '영업시간',
    'statusInfo_newOpenDate': '신규오픈날짜',
    'juso_roadAddrPart1': '도로명주소',
    'juso_siNm': '시',
    'juso_sggNm': '구',
    'juso_rn': '도로명',
    'bookStatus': '예약',
    'review_review': '리뷰_상세',
    'review_reviewSimple': '리뷰_요약',
    'review_readerAppraisal': '리뷰_평가',
    'etcInfo_seat': '좌석_수',
    'etcInfo_room': '룸_수',
    'etcInfo_chain': '체인점여부',
    'etcInfo_renewal': '리뉴얼여부',
    'etcInfo_head': '대표점여부',
    'etcInfo_branch': '지점여부'
}

# 3. 컬럼명 변경
restaurant_data.rename(columns=column_rename_mapping, inplace=True)

# 4. '예약' 컬럼 값 매핑
booking_status_mapping = {
    'APPROVAL': '예약가능',
    'INACTIVE': '예약불가능',
    'COVER': '정보 없음',
    'REJECT': '정보 없음',
    'HOLD': '정보 없음'
}
restaurant_data['예약'] = restaurant_data['예약'].str.strip()  # 공백 제거
restaurant_data['예약'] = restaurant_data['예약'].map(booking_status_mapping).fillna('정보 없음')

# 5. 결측값 처리 (예시: 필수 컬럼에 대해 기본값 지정)
restaurant_data.fillna({
    '리본개수수': '정보 없음',
    '전화번호': '미등록',
    '영업시간_평일': '미등록',
    '영업종료_평일': '미등록',
    '영업시간_주말': '미등록',
    '영업종료_주말': '미등록',
    '웹사이트': '미등록',
    '도로명주소': '미등록',
    '시': '미등록',
    '구': '미등록',
    '도로명': '미등록',
    '리뷰_상세': '정보 없음',
    '리뷰_요약': '정보 없음',
    '리뷰_평가': '정보 없음',
    '좌석_수': '미등록',
    '룸_수': '미등록',
    '체인점여부': '정보 없음',
    '리뉴얼여부': '정보 없음',
    '대표점여부': '정보 없음',
    '지점여부': '정보 없음'
}, inplace=True)

final_file_path = 'data/processed_restaurant_data_with_all_columns.csv'
restaurant_data.to_csv(final_file_path, index=False)




C:\Users\qkrck\AppData\Local\Temp\ipykernel_15476\3926361600.py:5: DtypeWarning: Columns (8,9,27) have mixed types. Specify dtype option on import or set low_memory=False.
  restaurant_data = pd.read_csv(file_path)


In [8]:
data = pd.read_csv("data/processed_restaurant_data_with_all_columns.csv")
data.head()

,id,예약,음식종류,식당이름,리본개수,전화번호,영업시간_평일,영업종료_평일,영업시간_주말,영업종료_주말,...,리뷰_상세,리뷰_요약,리뷰_평가,좌석_수,룸_수,체인점여부,리뉴얼여부,대표점여부,지점여부,웹사이트
0,4627,예약가능,"중식, 일반중식",홍보각,RIBBON_THREE,02-531-6479,미등록,미등록,미등록,미등록,...,국내 중식의 대가인 여경래 셰프가 2023년 이전하여 오픈한 중식 파인 다이닝 레스...,국내 중식의 대가인 여경래 셰프가 2023년 이전하여 오픈한 중식 파인 다이닝 레스...,정보 없음,64,"6개(2, 3, 4, 6, 8, 12석)",False,False,정보 없음,정보 없음,https://www.instagram.com/hongbogak_official/
1,29083,예약가능,"컨템포러리, 이노베이티브퀴진, 분자요리, 이탈리아식",쵸이닷,RIBBON_THREE,02-518-0318,12:00,22:30,미등록,미등록,...,최현석 셰프의 크리에이티브한 요리를 즐길 수 있는 레스토랑. 분자요리와 이탈리아 요...,최현석 셰프의 크리에이티브한 요리를 즐길 수 있는 레스토랑. 분자요리와 이탈리아 요...,“어디까지 발전할 수 있을지 진심으로 기대되는 곳.” “음식 너무나 재밌었어요.”,미등록,미등록,False,False,정보 없음,정보 없음,https://www.instagram.com/choidot_official/
2,28946,예약가능,프랑스식,더그린테이블,RIBBON_THREE,02-591-2672,미등록,미등록,미등록,미등록,...,김은희 셰프가 운영하는 프렌치 레스토랑. 사계절에 따라 변화하는 우리나라 식재료로 ...,김은희 셰프가 운영하는 프렌치 레스토랑. 사계절에 따라 변화하는 우리나라 식재료로 ...,“음식의 결 하나하나에서 섬세한 정성이 듬뿍 느껴진다. 신선한 재료들이 훌륭하게 조...,미등록,미등록,False,False,정보 없음,정보 없음,"http://thegreentable.co.kr/, https://www.insta..."
3,4963,예약가능,"한식(육류), 소갈비, 소고기구이, 한우오마카세",명월관,RIBBON_THREE,02-6330-9050,미등록,미등록,미등록,미등록,...,"워커힐호텔 내 한옥 분위기의 별채로 있는, 주변 경치가 아름다운 숯불구이 전문점. ...","워커힐호텔 내 한옥 분위기의 별채로 있는, 주변 경치가 아름다운 숯불구이 전문점. ...",정보 없음,300,4개,False,False,정보 없음,정보 없음,https://www.walkerhill.com/grandwalkerhillseou...
4,859,예약가능,프랑스식,콘티넨탈,RIBBON_THREE,02-2230-3369,미등록,미등록,미등록,미등록,...,"정통 유럽 스타일의 프렌치 레스토랑으로, 40여 년의 역사를 자랑하는, 서울에서 오...","정통 유럽 스타일의 프렌치 레스토랑으로, 40여 년의 역사를 자랑하는, 서울에서 오...",“최고의 레스토랑이다.” “분위기가 상당히 모던하고 세련된 느낌. 재료의 질이 최상...,70석,1개,False,False,정보 없음,정보 없음,https://www.shilla.net/seoul/dining/viewDining...


In [9]:
def preprocess_ribbon(row):
    if row == "RIBBON_ONE":
        return 1
    elif row == "RIBBON_TWO":
        return 2
    elif row == "RIBBON_THREE":
        return 3
    else:
        return 0  # NOT_RECORD

In [11]:

def preprocess_ribbon(row):
    if row == "RIBBON_ONE":
        return 1
    elif row == "RIBBON_TWO":
        return 2
    elif row == "RIBBON_THREE":
        return 3
    else:
        return 0  # NOT_RECORD

# 변환 수행
data["리본개수"] = data["리본개수"].apply(preprocess_ribbon)

# 결과 확인
print(data['리본개수'].value_counts())
data
# data["ribbon_count"] = data["리본개수"].apply(preprocessribbon)

# # "리본
# # 개수" 컬럼에서 1, 2, 3을 제외한 나머지 값을 0으로 변경
# data['리본개수'] = data['리본개수'].apply(lambda x: x if x in [1, 2, 3] else 0)

# # 변경된 데이터 확인
# print(data['리본개수'].value_counts())
# # "리본_개수" 컬럼의 고유 값 확인
# unique_values = data['리본개수'].unique()

# # 유효한 값 목록
# valid_values = {0, 1, 2, 3}

# # 값 검증
# invalid_values = set(unique_values) - valid_values

# if not invalid_values:
#     print("모든 '리본개수' 값이 0, 1, 2, 3에 해당합니다.")
# else:
#     print(f"유효하지 않은 값이 있습니다: {invalid_values}")

리본개수
0    17280
Name: count, dtype: int64


,id,예약,음식종류,식당이름,리본개수,전화번호,영업시간_평일,영업종료_평일,영업시간_주말,영업종료_주말,...,리뷰_상세,리뷰_요약,리뷰_평가,좌석_수,룸_수,체인점여부,리뉴얼여부,대표점여부,지점여부,웹사이트
0,4627,예약가능,"중식, 일반중식",홍보각,0,02-531-6479,미등록,미등록,미등록,미등록,...,국내 중식의 대가인 여경래 셰프가 2023년 이전하여 오픈한 중식 파인 다이닝 레스...,국내 중식의 대가인 여경래 셰프가 2023년 이전하여 오픈한 중식 파인 다이닝 레스...,정보 없음,64,"6개(2, 3, 4, 6, 8, 12석)",False,False,정보 없음,정보 없음,https://www.instagram.com/hongbogak_official/
1,29083,예약가능,"컨템포러리, 이노베이티브퀴진, 분자요리, 이탈리아식",쵸이닷,0,02-518-0318,12:00,22:30,미등록,미등록,...,최현석 셰프의 크리에이티브한 요리를 즐길 수 있는 레스토랑. 분자요리와 이탈리아 요...,최현석 셰프의 크리에이티브한 요리를 즐길 수 있는 레스토랑. 분자요리와 이탈리아 요...,“어디까지 발전할 수 있을지 진심으로 기대되는 곳.” “음식 너무나 재밌었어요.”,미등록,미등록,False,False,정보 없음,정보 없음,https://www.instagram.com/choidot_official/
2,28946,예약가능,프랑스식,더그린테이블,0,02-591-2672,미등록,미등록,미등록,미등록,...,김은희 셰프가 운영하는 프렌치 레스토랑. 사계절에 따라 변화하는 우리나라 식재료로 ...,김은희 셰프가 운영하는 프렌치 레스토랑. 사계절에 따라 변화하는 우리나라 식재료로 ...,“음식의 결 하나하나에서 섬세한 정성이 듬뿍 느껴진다. 신선한 재료들이 훌륭하게 조...,미등록,미등록,False,False,정보 없음,정보 없음,"http://thegreentable.co.kr/, https://www.insta..."
3,4963,예약가능,"한식(육류), 소갈비, 소고기구이, 한우오마카세",명월관,0,02-6330-9050,미등록,미등록,미등록,미등록,...,"워커힐호텔 내 한옥 분위기의 별채로 있는, 주변 경치가 아름다운 숯불구이 전문점. ...","워커힐호텔 내 한옥 분위기의 별채로 있는, 주변 경치가 아름다운 숯불구이 전문점. ...",정보 없음,300,4개,False,False,정보 없음,정보 없음,https://www.walkerhill.com/grandwalkerhillseou...
4,859,예약가능,프랑스식,콘티넨탈,0,02-2230-3369,미등록,미등록,미등록,미등록,...,"정통 유럽 스타일의 프렌치 레스토랑으로, 40여 년의 역사를 자랑하는, 서울에서 오...","정통 유럽 스타일의 프렌치 레스토랑으로, 40여 년의 역사를 자랑하는, 서울에서 오...",“최고의 레스토랑이다.” “분위기가 상당히 모던하고 세련된 느낌. 재료의 질이 최상...,70석,1개,False,False,정보 없음,정보 없음,https://www.shilla.net/seoul/dining/viewDining...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17275,25891,예약불가능,"한식(어패류), 생선회",8호단골횟집,0,032-885-9955,10:00,22:00,미등록,미등록,...,다양한 해산물을 저렴한 가격에 모두 맛볼 수 있는 곳. 스페셜 메뉴를 주문하면 한상...,다양한 해산물을 저렴한 가격에 모두 맛볼 수 있는 곳. 스페셜 메뉴를 주문하면 한상...,정보 없음,미등록,미등록,False,False,정보 없음,정보 없음,미등록
17276,26401,예약불가능,"일식, 스시",키무스시,0,031-411-0603,12:00,22:00,미등록,미등록,...,아담한 크기의 매장에서 스시를 즐길 수 있는 곳. 주문을 하면 일본인 주방장이 바로...,아담한 크기의 매장에서 스시를 즐길 수 있는 곳. 주문을 하면 일본인 주방장이 바로...,정보 없음,미등록,미등록,False,False,정보 없음,정보 없음,미등록
17277,26400,예약불가능,"카페/커피/티, 카페",카페바다,0,010-3713-3256,10:00,19:00,미등록,미등록,...,안면도 밧개 해변 근처에서 핸드드립커피를 즐길 수 있는 곳. 아기자기하고 아늑한 인...,안면도 밧개 해변 근처에서 핸드드립커피를 즐길 수 있는 곳. 아기자기하고 아늑한 인...,정보 없음,미등록,미등록,False,False,정보 없음,정보 없음,"https://cafebada.modoo.at/, https://www.instag..."
17278,26203,예약불가능,"디저트/베이커리, 떡",미시루,0,042-825-9902,06:00,19:00,미등록,미등록,...,"현대적 감각의 맛과 멋, 영양까지 골고루 갖춘 떡을 맛볼 수 있는 곳. 100% 천...","현대적 감각의 맛과 멋, 영양까지 골고루 갖춘 떡을 맛볼 수 있는 곳. 100% 천...",정보 없음,미등록,미등록,False,False,정보 없음,진관점(02-388-7710),http://www.미시루.com/


In [12]:
# 변경된 데이터를 새 CSV 파일로 저장
output_path = "data/final_restaurant.csv"
data.to_csv(output_path, index=False, encoding="utf-8-sig")  # UTF-8-sig로 저장하면 Excel 호환성 높음
print(f"변경된 데이터를 '{output_path}'에 저장했습니다.")

변경된 데이터를 'data/final_restaurant.csv'에 저장했습니다.


In [13]:
test = pd.read_csv("data/final_restaurant.csv")
test

,id,예약,음식종류,식당이름,리본개수,전화번호,영업시간_평일,영업종료_평일,영업시간_주말,영업종료_주말,...,리뷰_상세,리뷰_요약,리뷰_평가,좌석_수,룸_수,체인점여부,리뉴얼여부,대표점여부,지점여부,웹사이트
0,4627,예약가능,"중식, 일반중식",홍보각,0,02-531-6479,미등록,미등록,미등록,미등록,...,국내 중식의 대가인 여경래 셰프가 2023년 이전하여 오픈한 중식 파인 다이닝 레스...,국내 중식의 대가인 여경래 셰프가 2023년 이전하여 오픈한 중식 파인 다이닝 레스...,정보 없음,64,"6개(2, 3, 4, 6, 8, 12석)",False,False,정보 없음,정보 없음,https://www.instagram.com/hongbogak_official/
1,29083,예약가능,"컨템포러리, 이노베이티브퀴진, 분자요리, 이탈리아식",쵸이닷,0,02-518-0318,12:00,22:30,미등록,미등록,...,최현석 셰프의 크리에이티브한 요리를 즐길 수 있는 레스토랑. 분자요리와 이탈리아 요...,최현석 셰프의 크리에이티브한 요리를 즐길 수 있는 레스토랑. 분자요리와 이탈리아 요...,“어디까지 발전할 수 있을지 진심으로 기대되는 곳.” “음식 너무나 재밌었어요.”,미등록,미등록,False,False,정보 없음,정보 없음,https://www.instagram.com/choidot_official/
2,28946,예약가능,프랑스식,더그린테이블,0,02-591-2672,미등록,미등록,미등록,미등록,...,김은희 셰프가 운영하는 프렌치 레스토랑. 사계절에 따라 변화하는 우리나라 식재료로 ...,김은희 셰프가 운영하는 프렌치 레스토랑. 사계절에 따라 변화하는 우리나라 식재료로 ...,“음식의 결 하나하나에서 섬세한 정성이 듬뿍 느껴진다. 신선한 재료들이 훌륭하게 조...,미등록,미등록,False,False,정보 없음,정보 없음,"http://thegreentable.co.kr/, https://www.insta..."
3,4963,예약가능,"한식(육류), 소갈비, 소고기구이, 한우오마카세",명월관,0,02-6330-9050,미등록,미등록,미등록,미등록,...,"워커힐호텔 내 한옥 분위기의 별채로 있는, 주변 경치가 아름다운 숯불구이 전문점. ...","워커힐호텔 내 한옥 분위기의 별채로 있는, 주변 경치가 아름다운 숯불구이 전문점. ...",정보 없음,300,4개,False,False,정보 없음,정보 없음,https://www.walkerhill.com/grandwalkerhillseou...
4,859,예약가능,프랑스식,콘티넨탈,0,02-2230-3369,미등록,미등록,미등록,미등록,...,"정통 유럽 스타일의 프렌치 레스토랑으로, 40여 년의 역사를 자랑하는, 서울에서 오...","정통 유럽 스타일의 프렌치 레스토랑으로, 40여 년의 역사를 자랑하는, 서울에서 오...",“최고의 레스토랑이다.” “분위기가 상당히 모던하고 세련된 느낌. 재료의 질이 최상...,70석,1개,False,False,정보 없음,정보 없음,https://www.shilla.net/seoul/dining/viewDining...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17275,25891,예약불가능,"한식(어패류), 생선회",8호단골횟집,0,032-885-9955,10:00,22:00,미등록,미등록,...,다양한 해산물을 저렴한 가격에 모두 맛볼 수 있는 곳. 스페셜 메뉴를 주문하면 한상...,다양한 해산물을 저렴한 가격에 모두 맛볼 수 있는 곳. 스페셜 메뉴를 주문하면 한상...,정보 없음,미등록,미등록,False,False,정보 없음,정보 없음,미등록
17276,26401,예약불가능,"일식, 스시",키무스시,0,031-411-0603,12:00,22:00,미등록,미등록,...,아담한 크기의 매장에서 스시를 즐길 수 있는 곳. 주문을 하면 일본인 주방장이 바로...,아담한 크기의 매장에서 스시를 즐길 수 있는 곳. 주문을 하면 일본인 주방장이 바로...,정보 없음,미등록,미등록,False,False,정보 없음,정보 없음,미등록
17277,26400,예약불가능,"카페/커피/티, 카페",카페바다,0,010-3713-3256,10:00,19:00,미등록,미등록,...,안면도 밧개 해변 근처에서 핸드드립커피를 즐길 수 있는 곳. 아기자기하고 아늑한 인...,안면도 밧개 해변 근처에서 핸드드립커피를 즐길 수 있는 곳. 아기자기하고 아늑한 인...,정보 없음,미등록,미등록,False,False,정보 없음,정보 없음,"https://cafebada.modoo.at/, https://www.instag..."
17278,26203,예약불가능,"디저트/베이커리, 떡",미시루,0,042-825-9902,06:00,19:00,미등록,미등록,...,"현대적 감각의 맛과 멋, 영양까지 골고루 갖춘 떡을 맛볼 수 있는 곳. 100% 천...","현대적 감각의 맛과 멋, 영양까지 골고루 갖춘 떡을 맛볼 수 있는 곳. 100% 천...",정보 없음,미등록,미등록,False,False,정보 없음,진관점(02-388-7710),http://www.미시루.com/
